# Books recommendation system

Dataset used:
https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset?resource=download

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import defaultdict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OrdinalEncoder
from scipy.sparse import coo_matrix, csr_matrix, diags, linalg
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

np.random.seed(420)


## Dataset preparation

In [ ]:
df1 = pd.read_csv('Books.csv')

df1.head()

<ipython-input-2-bf431c85dabe>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('Books.csv')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
df1.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)

df1.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [ ]:
df2 = pd.read_csv('Ratings.csv')

df2.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
df3 = pd.read_csv('Users.csv')

df3.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
df3 = df3.fillna(0)

df3.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",0.0
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",0.0
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",0.0


In [ ]:
df = pd.merge(pd.merge(df1, df2, on = "ISBN"), df3, on = "User-ID")

df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Location,Age
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0,"stockton, california, usa",18.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,5,"timmins, ontario, canada",0.0
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,8,0,"timmins, ontario, canada",0.0
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,8,0,"timmins, ontario, canada",0.0
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,8,0,"timmins, ontario, canada",0.0


In [ ]:
df.dtypes

ISBN                    object
Book-Title              object
Book-Author             object
Year-Of-Publication     object
Publisher               object
User-ID                  int64
Book-Rating              int64
Location                object
Age                    float64
dtype: object

In [ ]:
len(df1), len(df2), len(df3), len(df)

(271360, 1149780, 278858, 1031136)

### Basic way

In [ ]:
categorical_columns = ['User-ID', 'Age', 'ISBN', 'Book-Title', 'Book-Author', 'Publisher', 'Location', 'Book-Rating']
ordinal_enc = OrdinalEncoder()

df_categorical = pd.DataFrame(np.array(ordinal_enc.fit_transform(df.loc[:, categorical_columns]),
                                           dtype=np.int64), columns=categorical_columns)
df_categorical.head()

,User-ID,Age,ISBN,Book-Title,Book-Author,Publisher,Location,Book-Rating
0,0,18,24927,36197,64911,10924,19186,0
1,1,0,73,36036,81125,6640,19911,5
2,1,0,8175,47448,12612,6643,19911,0
3,1,0,59963,68219,34145,5270,19911,0
4,1,0,71447,198739,24978,15773,19911,0


In [ ]:
for col in df_categorical.columns:
    print(col, ':', len(df_categorical[col].unique()))

User-ID : 92106
Age : 141
ISBN : 270151
Book-Title : 241071
Book-Author : 101589
Publisher : 16730
Location : 22480
Book-Rating : 11


In [ ]:
main_df = df_categorical
main_df.head(3)

,User-ID,Age,ISBN,Book-Title,Book-Author,Publisher,Location,Book-Rating
0,0,18,24927,36197,64911,10924,19186,0
1,1,0,73,36036,81125,6640,19911,5
2,1,0,8175,47448,12612,6643,19911,0


In [ ]:
df_train, df_test = train_test_split(main_df, random_state=420, test_size=0.2)

### ADM way

In [ ]:
pbr = df.groupby('Publisher')['Book-Rating'].mean()
s = pd.merge(df, pbr, on = "Publisher")
s.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating_x,Location,Age,Book-Rating_y
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0,"stockton, california, usa",18.0,3.496732
1,0198320264,Julius Caesar (Oxford School Shakespeare),William Shakespeare,2001,Oxford University Press,11676,0,"n/a, n/a, n/a",0.0,3.496732
2,0192815326,"Frankenstein: Or, the Modern Prometheus (World...",Mary Wollstonecraft Shelley,1982,Oxford University Press,11676,0,"n/a, n/a, n/a",0.0,3.496732
3,0192833375,Three Major Plays: Fuente Ovejuna/the Knight f...,Lope De Vega,1999,Oxford University Press,11676,0,"n/a, n/a, n/a",0.0,3.496732
4,019282760X,Pride and Prejudice (World's Classics),Jane Austen,1990,Oxford University Press,11676,9,"n/a, n/a, n/a",0.0,3.496732


In [ ]:
lbr = df.groupby('Location')['Book-Rating'].mean()
ss = pd.merge(s, lbr, on = 'Location')
ss.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating_x,Location,Age,Book-Rating_y,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0,"stockton, california, usa",18.0,3.496732,3.730612
1,0399145923,Carolina Moon,Nora Roberts,2000,Putnam Pub Group,86107,0,"stockton, california, usa",0.0,2.716981,3.730612
2,0425182908,Isle of Dogs,Patricia Cornwell,2002,Berkley Publishing Group,188593,0,"stockton, california, usa",26.0,2.424827,3.730612
3,0425184226,The Sum of All Fears,Tom Clancy,2002,Berkley Publishing Group,188593,0,"stockton, california, usa",26.0,2.424827,3.730612
4,042516098X,Hornet's Nest,Patricia Daniels Cornwell,1998,Berkley Publishing Group,188593,0,"stockton, california, usa",26.0,2.424827,3.730612


In [ ]:
ss.rename(columns={'Book-Rating_x': 'Book-Rating', 'Book-Rating_y': 'PBR', 'Book-Rating': 'LBR'}, inplace=True)
ss.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Location,Age,PBR,LBR
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0,"stockton, california, usa",18.0,3.496732,3.730612
1,0399145923,Carolina Moon,Nora Roberts,2000,Putnam Pub Group,86107,0,"stockton, california, usa",0.0,2.716981,3.730612
2,0425182908,Isle of Dogs,Patricia Cornwell,2002,Berkley Publishing Group,188593,0,"stockton, california, usa",26.0,2.424827,3.730612
3,0425184226,The Sum of All Fears,Tom Clancy,2002,Berkley Publishing Group,188593,0,"stockton, california, usa",26.0,2.424827,3.730612
4,042516098X,Hornet's Nest,Patricia Daniels Cornwell,1998,Berkley Publishing Group,188593,0,"stockton, california, usa",26.0,2.424827,3.730612


In [ ]:
bra = df.groupby('Book-Author')['Book-Rating'].mean()
mean_df = pd.merge(ss, bra, on = 'Book-Author')
mean_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating_x,Location,Age,PBR,LBR,Book-Rating_y
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0,"stockton, california, usa",18.0,3.496732,3.730612,3.50000
1,0801319536,Classical Mythology,Mark P. O. Morford,1998,John Wiley &amp; Sons,269782,7,"edmonton, alberta, canada",30.0,3.347619,2.731525,3.50000
2,0399145923,Carolina Moon,Nora Roberts,2000,Putnam Pub Group,86107,0,"stockton, california, usa",0.0,2.716981,3.730612,2.65939
3,0515135062,Three Fates,Nora Roberts,2004,Jove Books,242794,0,"stockton, california, usa",32.0,2.395111,3.730612,2.65939
4,0399145923,Carolina Moon,Nora Roberts,2000,Putnam Pub Group,11676,10,"n/a, n/a, n/a",0.0,2.716981,4.579018,2.65939


In [ ]:
mean_df.rename(columns={'Book-Rating_x': 'Book-Rating', 'Book-Rating_y': 'BRA'}, inplace=True)
mean_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Location,Age,PBR,LBR,BRA
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0,"stockton, california, usa",18.0,3.496732,3.730612,3.50000
1,0801319536,Classical Mythology,Mark P. O. Morford,1998,John Wiley &amp; Sons,269782,7,"edmonton, alberta, canada",30.0,3.347619,2.731525,3.50000
2,0399145923,Carolina Moon,Nora Roberts,2000,Putnam Pub Group,86107,0,"stockton, california, usa",0.0,2.716981,3.730612,2.65939
3,0515135062,Three Fates,Nora Roberts,2004,Jove Books,242794,0,"stockton, california, usa",32.0,2.395111,3.730612,2.65939
4,0399145923,Carolina Moon,Nora Roberts,2000,Putnam Pub Group,11676,10,"n/a, n/a, n/a",0.0,2.716981,4.579018,2.65939


In [ ]:
coords = mean_df[['User-ID', 'ISBN']].values
vals = mean_df['Book-Rating'].values

ucol = coords[:, 0]
bcol = coords[:, 1]

mucol = {thing: i for i,thing in enumerate(np.unique(ucol))}
mbcol = {thing: i for i,thing in enumerate(np.unique(bcol))}

nucol = np.array([mucol[a] for a in ucol])
nbcol = np.array([mbcol[a] for a in bcol])

cos_matrix = csr_matrix((vals, (nucol, nbcol)), shape=(len(mucol), len(mbcol)))

cos_matrix.shape

(92106, 270148)

In [ ]:
row_norms = linalg.norm(cos_matrix, axis=1)
normalized_matrix = cos_matrix.multiply(1 / (row_norms[:, np.newaxis]))

cosine_sim = normalized_matrix.dot(normalized_matrix.T)

<ipython-input-20-b82554950d89>:2: RuntimeWarning: divide by zero encountered in true_divide
  normalized_matrix = cos_matrix.multiply(1 / (row_norms[:, np.newaxis]))
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_compressed.py:470: RuntimeWarning: invalid value encountered in multiply
  data = np.multiply(ret.data, other[ret.row].ravel())


In [ ]:
print(cosine_sim)

  (0, 0)	nan
  (1, 74675)	0.06598795479344474
  (1, 7829)	nan
  (1, 18648)	nan
  (1, 50785)	nan
  (1, 55724)	nan
  (1, 87044)	0.03784080109036774
  (1, 72126)	0.05898804727117089
  (1, 69462)	0.14823417800568506
  (1, 65830)	0.2852126942296184
  (1, 40854)	0.3363363969981562
  (1, 38623)	0.04887390920459452
  (1, 22455)	0.12655946698124634
  (1, 13727)	nan
  (1, 3629)	0.004234516427955321
  (1, 1)	0.9999999999999999
  (2, 52983)	nan
  (2, 89662)	0.1448818969499362
  (2, 89655)	0.5598925109558544
  (2, 89200)	0.23863697995167382
  (2, 87944)	0.17908612711114452
  (2, 87390)	0.14436106547701633
  (2, 85410)	0.27146264489895156
  (2, 84580)	nan
  (2, 84057)	0.27934688485989034
  :	:
  (92105, 22102)	0.1293833631554033
  (92105, 21524)	0.04348208197492612
  (92105, 21249)	0.19735935605211033
  (92105, 20318)	nan
  (92105, 20271)	0.32549338848269405
  (92105, 19987)	0.4068667356033675
  (92105, 18492)	0.0790666237244296
  (92105, 15370)	0.018741351451440413
  (92105, 14859)	0.08238843851176

In [ ]:
coo = cosine_sim.tocoo()

In [ ]:
coo.setdiag(0)

In [ ]:
cos_values = {i: coo.col[coo.row == i][np.argmax(coo.data[coo.row == i])] for i in range(coo.shape[0])}

In [ ]:
rev_mucol = {thing: i for i,thing in mucol.items()}
decoded_cos_values = np.array([[rev_mucol[a], rev_mucol[b]] for a,b in cos_values.items()])

cos_df = pd.DataFrame(decoded_cos_values, columns=['User-ID', 'Cos-User-ID'])
cos_df.head()

,User-ID,Cos-User-ID
0,2,2
1,8,24539
2,9,160843
3,10,267172
4,12,64632


In [ ]:
adm_df = pd.merge(mean_df, cos_df, on = 'User-ID')
adm_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Location,Age,PBR,LBR,BRA,Cos-User-ID
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,2,0,"stockton, california, usa",18.0,3.496732,3.730612,3.500000,2
1,0801319536,Classical Mythology,Mark P. O. Morford,1998,John Wiley &amp; Sons,269782,7,"edmonton, alberta, canada",30.0,3.347619,2.731525,3.500000,249932
2,0671024256,On Writing,Stephen King,2001,Pocket,269782,0,"edmonton, alberta, canada",30.0,2.498671,2.731525,3.606287,249932
3,0684862719,Pay It Forward: A Novel,Catherine Ryan Hyde,2000,Simon &amp; Schuster,269782,8,"edmonton, alberta, canada",30.0,2.882148,2.731525,3.043321,249932
4,0140039589,Watership Down,Richard Adams,1974,Penguin Books,269782,10,"edmonton, alberta, canada",30.0,3.203898,2.731525,3.331828,249932


In [ ]:
adm_categorical_columns = ['User-ID', 'Age', 'Cos-User-ID', 'ISBN', 'Book-Title', 'Book-Author', 'Publisher', 'Location', 'PBR', 'LBR', 'BRA', 'Book-Rating']
adm_ordinal_enc = OrdinalEncoder()

adm_df_categorical = pd.DataFrame(np.array(adm_ordinal_enc.fit_transform(adm_df.loc[:, adm_categorical_columns]),
                                           dtype=np.int64), columns=adm_categorical_columns)
adm_df_categorical.head()

,User-ID,Age,Cos-User-ID,ISBN,Book-Title,Book-Author,Publisher,Location,PBR,LBR,BRA,Book-Rating
0,0,18,0,24927,36197,64910,10923,19186,1275,2138,3032,0
1,89191,30,26961,172352,36197,64910,7888,5938,1150,1307,3032,7
2,89191,30,26961,132990,133501,90802,11709,5938,512,1307,3151,0
3,89191,30,26961,148277,138566,13857,13651,5938,762,1307,2473,8
4,89191,30,26961,15190,230244,81019,11338,5938,1030,1307,2842,10


In [ ]:
for col in adm_df_categorical.columns:
    print(col, ':', len(adm_df_categorical[col].unique()))

User-ID : 92106
Age : 141
Cos-User-ID : 32298
ISBN : 270148
Book-Title : 241069
Book-Author : 101587
Publisher : 16728
Location : 22480
PBR : 2088
LBR : 3798
BRA : 4280
Book-Rating : 11


In [ ]:
adm_main_df = adm_df_categorical
adm_main_df.head(3)

,User-ID,Age,Cos-User-ID,ISBN,Book-Title,Book-Author,Publisher,Location,PBR,LBR,BRA,Book-Rating
0,0,18,0,24927,36197,64910,10923,19186,1275,2138,3032,0
1,89191,30,26961,172352,36197,64910,7888,5938,1150,1307,3032,7
2,89191,30,26961,132990,133501,90802,11709,5938,512,1307,3151,0


In [ ]:
adm_df_train, adm_df_test = train_test_split(adm_main_df, random_state=420, test_size=0.2)

## Dataset creation

In [ ]:
from torch.utils.data import Dataset

class Books(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        features = torch.tensor(self.data.iloc[index, :-1].values, dtype=torch.int64)
        target = torch.tensor(self.data.iloc[index, -1], dtype=torch.int64)
        return features, target

### Basic way

In [ ]:
test = Books(df_train)

for features, target in test:
    print(f"Features: {features}, Target: {target}")
    print(features[0], features[1])
    # print(test[0])
    print(df_train[df_train['User-ID']==64361])
    break

Features: tensor([ 64361,     29, 157000,  71223,  90803,  13320,  20205]), Target: 0
tensor(64361) tensor(29)
        User-ID  Age    ISBN  Book-Title  Book-Author  Publisher  Location  \
106087    64361   29  157000       71223        90803      13320     20205   
106117    64361   29   81652       47938        21432       1714     20205   
106393    64361   29  100670          22        22122      13594     20205   
105921    64361   29  118241        4229        86662       1414     20205   
106517    64361   29   37150      223270        94069      14027     20205   
...         ...  ...     ...         ...          ...        ...       ...   
106674    64361   29   92906        5525        24189      15861     20205   
105903    64361   29  165646       19157        40914       2311     20205   
106572    64361   29    9325      126451        93592       4996     20205   
106531    64361   29   95037      226669        62126       5285     20205   
106097    64361   29  139635   

### ADM way

In [ ]:
adm_test = Books(adm_df_train)

for features, target in adm_test:
    print(f"Features: {features}, Target: {target}")
    print(features[0], features[1])
    # print(test[0])
    print(adm_df_train[adm_df_train['User-ID']==64361])
    break

Features: tensor([ 89559,     53,  28193,  80296, 209698,  90802,   1714,  16937,    467,
          2833,   3151]), Target: 10
tensor(89559) tensor(53)
        User-ID  Age  Cos-User-ID    ISBN  Book-Title  Book-Author  Publisher  \
424155    64361   29        19248  158476      107148        12279       1129   
423973    64361   29        19248   93066       99972        42799      15860   
424093    64361   29        19248  191830       44130        15791       8565   
424320    64361   29        19248  169839      207967        16179       9809   
424194    64361   29        19248  121382      196822        22766       1418   
...         ...  ...          ...     ...         ...          ...        ...   
424424    64361   29        19248   11952      195441        11811       6610   
424158    64361   29        19248   93471      218687        24526      15860   
424211    64361   29        19248   64556       25565        16133      11424   
423962    64361   29        19248  100

## Model creation

### Basic way

In [ ]:
user_emb = 92106
age_emb = 141
ISBN_emb = 270151
book_title_emb = 241071
book_author_emb = 101589
publisher_emb = 16730
location_emb = 22480
book_rating_emb = 11

embedding_dim = 10

In [ ]:
class Recommender(nn.Module):
    def __init__(self, embedding_dim):
        super(Recommender, self).__init__()
        self.user_embedding = nn.Embedding(user_emb, embedding_dim)
        self.age_embedding = nn.Embedding(age_emb, embedding_dim)
        self.isbn_embedding = nn.Embedding(ISBN_emb, embedding_dim)
        self.title_embedding = nn.Embedding(book_title_emb, embedding_dim)
        self.author_embedding = nn.Embedding(book_author_emb, embedding_dim)
        self.publisher_embedding = nn.Embedding(publisher_emb, embedding_dim)
        self.location_embedding = nn.Embedding(location_emb, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 7, 88)
        self.fc2 = nn.Linear(88, 44)
        self.fc3 = nn.Linear(44, book_rating_emb)

    def forward(self, batch_input):
        user_embedded = self.user_embedding(batch_input[0])
        age_embedded = self.age_embedding(batch_input[1])
        isbn_embedded = self.isbn_embedding(batch_input[2])
        title_embedded = self.title_embedding(batch_input[3])
        author_embedded = self.author_embedding(batch_input[4])
        publisher_embedded = self.publisher_embedding(batch_input[5])
        location_embedded = self.location_embedding(batch_input[6])

        concatenated = torch.cat((user_embedded, age_embedded, isbn_embedded, title_embedded, author_embedded, publisher_embedded, location_embedded), dim=-1)
        x = torch.relu(self.fc1(concatenated))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

### ADM way

In [ ]:
user_emb = 92106
age_emb = 141
cos_user_emb = 32298
ISBN_emb = 270151
book_title_emb = 241071
book_author_emb = 101589
publisher_emb = 16730
location_emb = 22480
pbr_emb = 2088
lbr_emb = 3798
bra_emb = 4280
book_rating_emb = 11


embedding_dim = 10

In [ ]:
class ADM_Recommender(nn.Module):
    def __init__(self, embedding_dim):
        super(ADM_Recommender, self).__init__()
        self.user_embedding = nn.Embedding(user_emb, embedding_dim)
        self.age_embedding = nn.Embedding(age_emb, embedding_dim)
        self.cos_user_embedding = nn.Embedding(cos_user_emb, embedding_dim)
        self.isbn_embedding = nn.Embedding(ISBN_emb, embedding_dim)
        self.title_embedding = nn.Embedding(book_title_emb, embedding_dim)
        self.author_embedding = nn.Embedding(book_author_emb, embedding_dim)
        self.publisher_embedding = nn.Embedding(publisher_emb, embedding_dim)
        self.location_embedding = nn.Embedding(location_emb, embedding_dim)
        self.publisherbookrating_embedding = nn.Embedding(pbr_emb, embedding_dim)
        self.locationbookrating_embedding = nn.Embedding(lbr_emb, embedding_dim)
        self.bookratingauthor_embedding = nn.Embedding(bra_emb, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 11, 121)
        self.fc2 = nn.Linear(121, 66)
        self.fc3 = nn.Linear(66, book_rating_emb)

    def forward(self, batch_input):
        user_embedded = self.user_embedding(batch_input[0])
        age_embedded = self.age_embedding(batch_input[1])
        cos_user_embedded = self.cos_user_embedding(batch_input[2])
        isbn_embedded = self.isbn_embedding(batch_input[3])
        title_embedded = self.title_embedding(batch_input[4])
        author_embedded = self.author_embedding(batch_input[5])
        publisher_embedded = self.publisher_embedding(batch_input[6])
        location_embedded = self.location_embedding(batch_input[7])
        pbr_embedded = self.publisherbookrating_embedding(batch_input[8])
        lbr_embedded = self.locationbookrating_embedding(batch_input[9])
        bra_embedded = self.bookratingauthor_embedding(batch_input[10])

        concatenated = torch.cat((user_embedded, age_embedded, cos_user_embedded, isbn_embedded, title_embedded, author_embedded, publisher_embedded, location_embedded, pbr_embedded, lbr_embedded, bra_embedded), dim=-1)
        x = torch.relu(self.fc1(concatenated))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

## Model training

In [ ]:
def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

In [ ]:
num_epochs = 1
batch_size = 200
learning_rate = 0.1
weight_decay = 0.001

### Basic way

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

model = Recommender(embedding_dim)
model = model.to(device)

loss_criterion = torch.nn.CrossEntropyLoss(reduction="mean")

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay,
)

dataset = Books(df_train)

step = 0
tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor

for epoch in range(num_epochs):

    data_loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
    )

    loss_tracker = defaultdict(tensor)

    model.train()

    for batch in data_loader:
        for i, b in enumerate(batch):
            if torch.is_tensor(b[0]):
                b[0] = to_var(b[0])

            preds = model(batch[0][i])

            loss = loss_criterion(preds, batch[1][i])

            optimizer.zero_grad()
            loss.backward()
        optimizer.step()
        step += 1

        loss_tracker["Total Loss"] = torch.cat((loss_tracker["Total Loss"], loss.view(1)))

    print("Mean total loss: ",torch.mean(loss_tracker["Total Loss"]))



Mean total loss:  tensor(1.9442, grad_fn=<MeanBackward0>)


### ADM way

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

adm_model = ADM_Recommender(embedding_dim)
adm_model = adm_model.to(device)

loss_criterion = torch.nn.CrossEntropyLoss(reduction="mean")

optimizer = torch.optim.Adam(
    adm_model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay,
)

dataset = Books(adm_df_train)

step = 0
tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor

for epoch in range(num_epochs):

    data_loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
    )

    loss_tracker = defaultdict(tensor)

    adm_model.train()

    for batch in data_loader:
        for i, b in enumerate(batch):
            if torch.is_tensor(b[0]):
                b[0] = to_var(b[0])

            preds = adm_model(batch[0][i])
            loss = loss_criterion(preds, batch[1][i])

            optimizer.zero_grad()
            loss.backward()
        optimizer.step()
        step += 1

        loss_tracker["Total Loss"] = torch.cat((loss_tracker["Total Loss"], loss.view(1)))

    print("Mean total loss: ",torch.mean(loss_tracker["Total Loss"]))



Mean total loss:  tensor(1.9695, grad_fn=<MeanBackward0>)


## Tests

In [ ]:
def compute_metrics(target, pred_probs):
    """
    Computes metrics to report
    """
    pred_labels = pred_probs.argmax(-1)
    precision = metrics.precision_score(target, pred_labels, average="macro")
    recall = metrics.recall_score(target, pred_labels, average="macro")
    f1_score = metrics.f1_score(target, pred_labels, average="macro")
    accuracy = metrics.accuracy_score(target, pred_labels)
    auc = metrics.roc_auc_score(target, pred_probs, average="macro", multi_class="ovr")

    return precision, recall, f1_score, accuracy, auc

### Test dataset

#### Basic way

In [ ]:
target_tracker = []
pred_tracker = []

dataset = Books(df_test)
data_loader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=False,
)

model.eval()
with torch.no_grad():
    for batch in data_loader:
        for i, b in enumerate(batch):
            if torch.is_tensor(b[0]):
                b[0] = to_var(b[0])

            preds = model(batch[0][i])
            pred_probs = F.softmax(preds, dim=-1)

            target_tracker.append(batch[1][i].cpu().numpy())
            pred_tracker.append(pred_probs.cpu().data.numpy())


target_tracker = np.stack(target_tracker[:-1]).reshape(-1)
pred_tracker = np.stack(pred_tracker[:-1], axis=0).reshape(-1,book_rating_emb)
precision, recall, f1_score, accuracy, auc = compute_metrics(target_tracker, pred_tracker)

print("Metrics:\n Precision = {:.3f}\n Recall = {:.3f}\n F1-score = {:.3f}\n Accuracy = {:.3f}\n AUC = {:.3f}\n ".format(precision, recall, f1_score, accuracy, auc))


Metrics:
 Precision = 0.009
 Recall = 0.091
 F1-score = 0.016
 Accuracy = 0.098
 AUC = 0.452
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### ADM way

In [ ]:
target_tracker = []
pred_tracker = []

dataset = Books(adm_df_test)
data_loader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=False,
)

adm_model.eval()
with torch.no_grad():
    for batch in data_loader:
        for i, b in enumerate(batch):
            if torch.is_tensor(b[0]):
                b[0] = to_var(b[0])

            preds = adm_model(batch[0][i])
            pred_probs = F.softmax(preds, dim=-1)

            target_tracker.append(batch[1][i].cpu().numpy())
            pred_tracker.append(pred_probs.cpu().data.numpy())


target_tracker = np.stack(target_tracker[:-1]).reshape(-1)
pred_tracker = np.stack(pred_tracker[:-1], axis=0).reshape(-1,book_rating_emb)
precision, recall, f1_score, accuracy, auc = compute_metrics(target_tracker, pred_tracker)

print("Metrics:\n Precision = {:.3f}\n Recall = {:.3f}\n F1-score = {:.3f}\n Accuracy = {:.3f}\n AUC = {:.3f}\n ".format(precision, recall, f1_score, accuracy, auc))


Metrics:
 Precision = 0.057
 Recall = 0.091
 F1-score = 0.070
 Accuracy = 0.622
 AUC = 0.468
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Full dataset

#### Basic way

In [ ]:
target_tracker = []
pred_tracker = []

dataset = Books(main_df)
data_loader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=False,
)

model.eval()
with torch.no_grad():
    for batch in data_loader:
        for i, b in enumerate(batch):
            if torch.is_tensor(b[0]):
                b[0] = to_var(b[0])

            preds = model(batch[0][i])
            pred_probs = F.softmax(preds, dim=-1)

            target_tracker.append(batch[1][i].cpu().numpy())
            pred_tracker.append(pred_probs.cpu().data.numpy())


target_tracker = np.stack(target_tracker[:-1]).reshape(-1)
pred_tracker = np.stack(pred_tracker[:-1], axis=0).reshape(-1,book_rating_emb)
precision, recall, f1_score, accuracy, auc = compute_metrics(target_tracker, pred_tracker)

print("Metrics:\n Precision = {:.3f}\n Recall = {:.3f}\n F1-score = {:.3f}\n Accuracy = {:.3f}\n AUC = {:.3f}\n ".format(precision, recall, f1_score, accuracy, auc))


Metrics:
 Precision = 0.008
 Recall = 0.091
 F1-score = 0.015
 Accuracy = 0.088
 AUC = 0.473
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### ADM way

In [ ]:
target_tracker = []
pred_tracker = []

dataset = Books(adm_main_df)
data_loader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=False,
)

adm_model.eval()
with torch.no_grad():
    for batch in data_loader:
        for i, b in enumerate(batch):
            if torch.is_tensor(b[0]):
                b[0] = to_var(b[0])

            preds = adm_model(batch[0][i])
            pred_probs = F.softmax(preds, dim=-1)

            target_tracker.append(batch[1][i].cpu().numpy())
            pred_tracker.append(pred_probs.cpu().data.numpy())


target_tracker = np.stack(target_tracker[:-1]).reshape(-1)
pred_tracker = np.stack(pred_tracker[:-1], axis=0).reshape(-1,book_rating_emb)
precision, recall, f1_score, accuracy, auc = compute_metrics(target_tracker, pred_tracker)

print("Metrics:\n Precision = {:.3f}\n Recall = {:.3f}\n F1-score = {:.3f}\n Accuracy = {:.3f}\n AUC = {:.3f}\n ".format(precision, recall, f1_score, accuracy, auc))


Metrics:
 Precision = 0.057
 Recall = 0.091
 F1-score = 0.070
 Accuracy = 0.629
 AUC = 0.507
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Different datasets

In [ ]:
model_seeds = [42, 69]
dataset_seeds = [210, 130, 8888]

#### Basic Way

In [ ]:
for seed in dataset_seeds:
    df_train, df_test = train_test_split(main_df, random_state=seed, test_size=0.3)
    target_tracker = []
    pred_tracker = []

    dataset = Books(df_test)
    data_loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
    )

    model.eval()
    with torch.no_grad():
        for batch in data_loader:
            for i, b in enumerate(batch):
                if torch.is_tensor(b[0]):
                    b[0] = to_var(b[0])

                preds = model(batch[0][i])
                pred_probs = F.softmax(preds, dim=-1)

                target_tracker.append(batch[1][i].cpu().numpy())
                pred_tracker.append(pred_probs.cpu().data.numpy())


    target_tracker = np.stack(target_tracker[:-1]).reshape(-1)
    pred_tracker = np.stack(pred_tracker[:-1], axis=0).reshape(-1,book_rating_emb)
    precision, recall, f1_score, accuracy, auc = compute_metrics(target_tracker, pred_tracker)

    print("Seed: {:.3f}, metrics:\n Precision = {:.3f}\n Recall = {:.3f}\n F1-score = {:.3f}\n Accuracy = {:.3f}\n AUC = {:.3f}\n ".format(seed, precision, recall, f1_score, accuracy, auc))


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seed: 210.000, metrics:
 Precision = 0.008
 Recall = 0.091
 F1-score = 0.015
 Accuracy = 0.090
 AUC = 0.444
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seed: 130.000, metrics:
 Precision = 0.009
 Recall = 0.091
 F1-score = 0.016
 Accuracy = 0.095
 AUC = 0.469
 
Seed: 8888.000, metrics:
 Precision = 0.007
 Recall = 0.091
 F1-score = 0.014
 Accuracy = 0.082
 AUC = 0.482
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### ADM way

In [ ]:
for seed in dataset_seeds:
    adm_df_train, adm_df_test = train_test_split(adm_main_df, random_state=seed, test_size=0.3)

    target_tracker = []
    pred_tracker = []

    dataset = Books(adm_df_test)
    data_loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
    )

    adm_model.eval()
    with torch.no_grad():
        for batch in data_loader:
            for i, b in enumerate(batch):
                if torch.is_tensor(b[0]):
                    b[0] = to_var(b[0])

                preds = adm_model(batch[0][i])
                pred_probs = F.softmax(preds, dim=-1)

                target_tracker.append(batch[1][i].cpu().numpy())
                pred_tracker.append(pred_probs.cpu().data.numpy())


    target_tracker = np.stack(target_tracker[:-1]).reshape(-1)
    pred_tracker = np.stack(pred_tracker[:-1], axis=0).reshape(-1,book_rating_emb)
    precision, recall, f1_score, accuracy, auc = compute_metrics(target_tracker, pred_tracker)

    print("Dataset seed: {:.3f}, metrics:\n Precision = {:.3f}\n Recall = {:.3f}\n F1-score = {:.3f}\n Accuracy = {:.3f}\n AUC = {:.3f}\n ".format(seed, precision, recall, f1_score, accuracy, auc))


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Dataset seed: 210.000, metrics:
 Precision = 0.056
 Recall = 0.091
 F1-score = 0.070
 Accuracy = 0.619
 AUC = 0.531
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Dataset seed: 130.000, metrics:
 Precision = 0.058
 Recall = 0.091
 F1-score = 0.070
 Accuracy = 0.633
 AUC = 0.513
 
Dataset seed: 8888.000, metrics:
 Precision = 0.058
 Recall = 0.091
 F1-score = 0.071
 Accuracy = 0.639
 AUC = 0.516
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Different model seeds and datasets

#### Basic way

In [ ]:
for seed in model_seeds:
    for dat_seed in dataset_seeds:
        df_train, df_test = train_test_split(main_df, random_state=dat_seed, test_size=0.3)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        torch.manual_seed(seed)

        model = Recommender(embedding_dim)
        model = model.to(device)

        loss_criterion = torch.nn.CrossEntropyLoss(reduction="mean")

        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=learning_rate,
            weight_decay=weight_decay,
        )

        dataset = Books(df_train)

        step = 0
        tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor

        for epoch in range(num_epochs):

            data_loader = DataLoader(
                dataset=dataset,
                batch_size=batch_size,
                shuffle=False,
            )

            loss_tracker = defaultdict(tensor)

            model.train()

            for batch in data_loader:
                for i, b in enumerate(batch):
                    if torch.is_tensor(b[0]):
                        b[0] = to_var(b[0])

                    preds = model(batch[0][i])

                    loss = loss_criterion(preds, batch[1][i])

                    optimizer.zero_grad()
                    loss.backward()
                optimizer.step()
                step += 1

                loss_tracker["Total Loss"] = torch.cat((loss_tracker["Total Loss"], loss.view(1)))

        target_tracker = []
        pred_tracker = []

        dataset = Books(df_test)
        data_loader = DataLoader(
            dataset=dataset,
            batch_size=batch_size,
            shuffle=False,
        )

        model.eval()
        with torch.no_grad():
            for batch in data_loader:
                for i, b in enumerate(batch):
                    if torch.is_tensor(b[0]):
                        b[0] = to_var(b[0])

                    preds = model(batch[0][i])
                    pred_probs = F.softmax(preds, dim=-1)

                    target_tracker.append(batch[1][i].cpu().numpy())
                    pred_tracker.append(pred_probs.cpu().data.numpy())


        target_tracker = np.stack(target_tracker[:-1]).reshape(-1)
        pred_tracker = np.stack(pred_tracker[:-1], axis=0).reshape(-1,book_rating_emb)
        precision, recall, f1_score, accuracy, auc = compute_metrics(target_tracker, pred_tracker)

        print("Seed: {:.3f}, dataset seed: {:.3f}, metrics:\n Precision = {:.3f}\n Recall = {:.3f}\n F1-score = {:.3f}\n Accuracy = {:.3f}\n AUC = {:.3f}\n ".format(seed, dat_seed, precision, recall, f1_score, accuracy, auc))


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seed: 42.000, dataset seed: 210.000, metrics:
 Precision = 0.006
 Recall = 0.091
 F1-score = 0.011
 Accuracy = 0.067
 AUC = 0.475
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seed: 42.000, dataset seed: 130.000, metrics:
 Precision = 0.006
 Recall = 0.091
 F1-score = 0.011
 Accuracy = 0.062
 AUC = 0.521
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seed: 69.000, dataset seed: 210.000, metrics:
 Precision = 0.058
 Recall = 0.091
 F1-score = 0.071
 Accuracy = 0.634
 AUC = 0.475
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seed: 69.000, dataset seed: 130.000, metrics:
 Precision = 0.056
 Recall = 0.091
 F1-score = 0.069
 Accuracy = 0.613
 AUC = 0.507
 
Seed: 69.000, dataset seed: 8888.000, metrics:
 Precision = 0.057
 Recall = 0.091
 F1-score = 0.070
 Accuracy = 0.624
 AUC = 0.506
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### ADM way

In [ ]:
for seed in model_seeds:
    for dat_seed in dataset_seeds:
        adm_df_train, adm_df_test = train_test_split(adm_main_df, random_state=dat_seed, test_size=0.3)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        torch.manual_seed(seed)

        adm_model = ADM_Recommender(embedding_dim)
        adm_model = adm_model.to(device)

        loss_criterion = torch.nn.CrossEntropyLoss(reduction="mean")

        optimizer = torch.optim.Adam(
            adm_model.parameters(),
            lr=learning_rate,
            weight_decay=weight_decay,
        )

        dataset = Books(adm_df_train)

        step = 0
        tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor

        for epoch in range(num_epochs):

            data_loader = DataLoader(
                dataset=dataset,
                batch_size=batch_size,
                shuffle=False,
            )

            loss_tracker = defaultdict(tensor)

            adm_model.train()

            for batch in data_loader:
                for i, b in enumerate(batch):
                    if torch.is_tensor(b[0]):
                        b[0] = to_var(b[0])

                    preds = adm_model(batch[0][i])
                    loss = loss_criterion(preds, batch[1][i])

                    optimizer.zero_grad()
                    loss.backward()
                optimizer.step()
                step += 1

                loss_tracker["Total Loss"] = torch.cat((loss_tracker["Total Loss"], loss.view(1)))


        target_tracker = []
        pred_tracker = []

        dataset = Books(adm_df_test)
        data_loader = DataLoader(
            dataset=dataset,
            batch_size=batch_size,
            shuffle=False,
        )

        adm_model.eval()
        with torch.no_grad():
            for batch in data_loader:
                for i, b in enumerate(batch):
                    if torch.is_tensor(b[0]):
                        b[0] = to_var(b[0])

                    preds = adm_model(batch[0][i])
                    pred_probs = F.softmax(preds, dim=-1)

                    target_tracker.append(batch[1][i].cpu().numpy())
                    pred_tracker.append(pred_probs.cpu().data.numpy())


        target_tracker = np.stack(target_tracker[:-1]).reshape(-1)
        pred_tracker = np.stack(pred_tracker[:-1], axis=0).reshape(-1,book_rating_emb)
        precision, recall, f1_score, accuracy, auc = compute_metrics(target_tracker, pred_tracker)

        print("Seed: {:.3f}, dataset seed: {:.3f}, metrics:\n Precision = {:.3f}\n Recall = {:.3f}\n F1-score = {:.3f}\n Accuracy = {:.3f}\n AUC = {:.3f}\n ".format(seed, dat_seed, precision, recall, f1_score, accuracy, auc))


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seed: 42.000, dataset seed: 210.000, metrics:
 Precision = 0.056
 Recall = 0.091
 F1-score = 0.070
 Accuracy = 0.619
 AUC = 0.496
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seed: 42.000, dataset seed: 130.000, metrics:
 Precision = 0.008
 Recall = 0.091
 F1-score = 0.015
 Accuracy = 0.093
 AUC = 0.514
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seed: 69.000, dataset seed: 210.000, metrics:
 Precision = 0.056
 Recall = 0.091
 F1-score = 0.070
 Accuracy = 0.619
 AUC = 0.520
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seed: 69.000, dataset seed: 130.000, metrics:
 Precision = 0.058
 Recall = 0.091
 F1-score = 0.070
 Accuracy = 0.633
 AUC = 0.470
 
Seed: 69.000, dataset seed: 8888.000, metrics:
 Precision = 0.058
 Recall = 0.091
 F1-score = 0.071
 Accuracy = 0.639
 AUC = 0.490
 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
